![Logo](imagenes/banner.png)

# **MEMORIA PROYECTO IA GENERATIVA APLICADO CON APIs**

El objetivo del proyecto es el de crear una aplicación de consultas mediante chat con una IA generativa. 

El tema elegido es un organizador de fiestas temáticas. AI.Fiesta.

* Para ello nos conectaremos a una de las IAs LLM disponibles mediante su API (gpt de openai), donde le indicaremos el rol que tiene y le daremos instrucciones de la forma con la que tiene que responder elaborando un prompt.
* También crearemos una base de datos en AWS donde se almacenen esos chats.
* Para el backend crearemos una API que conecte la API de la IA a nuestra base de datos mediante endpoints y a su vez con la interfaz.
* Todo esto se verá reflejado en un frontend donde el usuario interactuará con la IA para obtener la información deseada.
* Por último, dockerizaremos el proyecto, encapsulándolo en una imagen con Docker.


![Logo](imagenes/esquema_blanco.png)

In [1]:
# LIBRERIAS QUE UTILIZAMOS:
from flask import request, Flask, jsonify
import json
import psycopg2
import os
from datetime import datetime
from groq import Groq

## **BASE DE DATOS Y CONEXIÓN A AURORA & RDS (AWS)** 

Creamos una base de datos en AWS con motor PostgreSQL y la conectamos a la nuestra a través de pgAdmin 4. Creamos nuestra base de datos en PostgreSQL. Aquí también comprobaremos que las preguntas de los usuarios, las respuestas y la fecha de esas preguntas se van guardando correctamente.

In [ ]:
# Creamos la base de datos en PostgreSQL

"""
CREATE TABLE preguntas_respuestas(
    id SERIAL PRIMARY KEY,
    p_tema TEXT NOT NULL,
    edad_invitados TEXT NOT NULL,
    numero_invitados INTEGER,
    presupuesto TEXT NOT NULL,
    lugar TEXT NOT NULL,
    fecha TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    r_tema TEXT NOT NULL,
    r_musica TEXT [] NOT NULL,
    r_decoracion TEXT [] NOT NULL,
    r_juegos TEXT [] NOT NULL,
    r_comida TEXT [] NOT NULL,
    r_bebidas TEXT [] NOT NULL
    );
"""

In [ ]:
os.environ["PASSWORD_AWS"]= "****"
os.environ["HOST_AWS"]= "****.rds.amazonaws.com"

In [3]:
config = {
    "host":os.environ.get("HOST_AWS"),
    "user":"postgres",
    "password":os.environ.get("PASSWORD_AWS"),
    "port": 5432,
    "dbname": "postgres"
}

In [4]:
# Nos conectamos a la base de datos:
try:
    
    con = psycopg2.connect(**config)
    print("conexión establecida")
    con.close()
except Exception as e:
    print("conexión no establecida, algo falla")

conexión establecida


In [5]:
# Creamos cursor:

def bbdd(p_tema,edad_invitados,numero_invitados,presupuesto,lugar,r_tema,r_musica,r_decoracion,r_juegos,r_comida,r_bebidas):

    conn = psycopg2.connect(**config)
    cursor= conn.cursor()

    query = """INSERT INTO preguntas_respuestas (p_tema, edad_invitados, numero_invitados, presupuesto, lugar, fecha,
            r_tema, r_musica, r_decoracion, r_juegos, r_comida, r_bebidas) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"""

    cursor.execute(query,(p_tema, edad_invitados, numero_invitados, presupuesto, lugar, datetime.now(),
        r_tema, r_musica, r_decoracion, r_juegos, r_comida, r_bebidas))
    
    conn.commit()
    cursor.close()
    conn.close()

    return "ok"
   

--------------------------------------------------------

## **CONEXIÓN API GROQ PARA LLM** 

In [ ]:
os.environ['KEY_GROQ'] = "****"

In [ ]:
# FIESTAS TEMATICAS

def llm(p_tema,edad_invitados,numero_invitados,presupuesto,lugar):
    
    client = Groq(
        api_key=os.environ.get("KEY_GROQ"),
    )

    system_prompt = """
            Eres un organizador de fiestas temáticas. 
            Tu misión es crear planes completos de fiestas adaptados a la temática, la edad de los invitados y el número de personas.

            Reglas:
            - SOLO puedes responder sobre fiestas, celebraciones y organización de eventos. Responde en castellano de España.
            - Si el usuario pregunta por otro tema, responde:
            "Solo puedo ayudarte a organizar fiestas temáticas. ¿Quieres que te sugiera ideas para una celebración?"
            - LÍMITE: máximo 4 elementos en cada lista y solo strings (sin descripciones largas).
            - Debes devolver SIEMPRE un JSON válido con esta estructura:

            {
            "música": ["string", "..."],
            "decoración": ["string", "..."],
            "juegos": ["string", "..."],
            "comida": ["string", "..."],
            "bebidas": ["string", "..."]
            }

            Detalles:
            - Ajusta la música, juegos y decoración a la temática. Se riguroso con la música y las actividades sugeridas que sean de la temática.
            - La comida y las bebidas deben ser adecuadas a la edad de los invitados. Si es de adultos, sugiere 1 coctel. Si es de niños o adolescentes, nada de alcholo
            - Escala las sugerencias según el número de invitados (ej. cantidad de comida o tipo de juegos).
            - Responde SOLO con json (sin texto fuera del json)


            Ejemplo de salida:
                    {
            "tema": "Años 80",
            "música": [ "Queen", "Glam Metal", "Madonna"
            ],
            "decoración": [
                "Globos de neón",
                "Carteles retro ochenteros"
            ],
            "juegos": [
                "Concurso de baile estilo ochentero",
                "Trivia musical de los 80"
            ],
            "comida": [
                "Mini hamburguesas",
                "Palomitas de colores"
            ],
            "bebidas": [
                "Cóctel Blue Lagoon",
                "Refrescos clásicos (Coca-Cola en botellas de vidrio)"
            ]
            }
        """
    
    user_prompt = f"""
        Temática: {p_tema}
        Edad de invitados: {edad_invitados}
        Número de invitados: {numero_invitados}
        Presupuesto: {presupuesto}
        Lugar: {lugar}
        """.strip()

    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": system_prompt
            },
            {
                "role":"user",
                "content": user_prompt
            }
        ],
        model="openai/gpt-oss-20b",
        max_tokens=1500,
        temperature=0.2,
        response_format={"type": "json_object"},
        stream=False,
    )

    text = chat_completion.choices[0].message.content
    data = json.loads(text)
    return data

In [10]:
# Checkeo de que funciona

plan = llm("Años 80", "adolescentes", 10, "alto", "pub")

r_tema       = plan.get("tema", "Años 80")
r_musica     = plan.get("música", [])
r_decoracion = plan.get("decoración", [])
r_juegos     = plan.get("juegos", [])
r_comida     = plan.get("comida", [])
r_bebidas    = plan.get("bebidas", [])


resultado = bbdd("Años 80", "adolescentes", 10, "alto", "pub",
                 r_tema, r_musica, r_decoracion, r_juegos, r_comida, r_bebidas)

print("ok" if resultado == "ok" else "Error en la inserción")


#y efectivamente aparece en nuestra base de datos.

ok


## **CREAMOS LOS ARCHIVOS PARA UTILIZARLOS EN NUESTRA API** 


* *Memoria.ipynb*: Este archivo, donde documentamos y explicamos el proyecto
* *app.py*: archivo principal donde creamos nuestra API con Flask
* *funciones.py*: archivo con las funciones que hemos creado
* *.env*: donde almacenamos la información sensible (local)
* *variables.py*: archivo con las variables necesarias (excepto las sensibles)
* *index.html*: archivo con la interfaz de nuestra app
* *requeriments.txt*: archivo donde enumeramos las librerias necesarias para ejecutar el proyecto
* *Dockerfile*: archivo con instrucciones para crear la imagen que encapsula nuestro proyecto
* *test.py*: archivo con los test (pytest) para verificar que la API funciona correctamente


EJEMPLO DE ARCHIVO .env (no subido al repositorio por seguridad)
* PASSWORD_AWS= contraseña base de datos aws
* HOST_AWS= ****.rds.amazonaws.com
* KEY_GROQ = API KEY suministrada por GROQ para el proyecto

## **VISTA DEL FRONT** 
![Logo](imagenes/Web.png)

## **ALGUNAS POSIBLES MEJORAS DEL PROYECTO** 
* Mejora estética de la web.

En las respuestas: 
* Enlaces a playlists
* Moodboard con ideas de decoración
* Enlaces a juegos
* Enlaces a recetas de cocina y bebidas

En la base de datos:
* Ampliar tablas de la base de datos y dividir cada sugerencia

En el modelo LLM:
* Mejora de prompt para que las respuestas sean más concretas y centradas a la pregunta.
* Cambio de modelo por uno más potente
